# Forecasting COVID-19 Pandemic in Mozambique and Estimating Possible Scenarios

Our code does not retrived data from online repositories automatically, therefore, the user must eithre include a routine that does it, or download it's own data file in a .txt or .csv file in order to use the PANDAS library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
from scipy.integrate import odeint
from sklearn.metrics import r2_score

MoçM = pd.read_csv("computer address to file")#file on cumulative number of deaths each day
MoçR = pd.read_csv("computer address to file")#file on cumulative number of recoveries each day
MoçCT = pd.read_csv("computer address to file")#file on cumulative number of cases each day
MoçI = MoçCT.values.ravel() - MoçM.values.ravel() - MoçR.values.ravel()#Creating the current infected data using
#the cumulative number of cases - cumulative number of deaths - cumulative number of recoveries

y1 = MoçI
comboY = y1

t2 = np.linspace(1, len(MoçI), len(MoçI))
comboX = t2

Nmoç = 8975591*0.01

#Mod1 function will be used for data fitting
def mod1(t, b1, E0, I0):
    def Corona(z,t):
        S = z[0]
        E = z[1]
        I = z[2]
        R = z[3]
        M = z[4]
        Pq = 0.5
        b = Pq*(b1/N)/(1+0.01*np.exp(t-8)) + (1-Pq)*(b1/N)
        c = 1/5.1
        k = 0.44
        Pm = 0.007
        tr = 16
        a = (1-Pm)/tr
        tm = 13
        mu = Pm/tm
        dSdt = -(1-k)*b*I*S - k*b*E*S
        dEdt = (1-k)*b*I*S - c*E + k*b*E*S
        dIdt = c*E - a*I - mu*I
        dRdt = a*I
        dMdt = mu*I
        return [dSdt, dEdt, dIdt, dRdt, dMdt]
    
    
    #Defining initial conditions
    z0 = [Nmoç,E0,I0,0,0]
    y1 = odeint(Corona, z0, t)
    
    S = y1[:,0]
    E = y1[:,1]
    I = y1[:,2]
    R = y1[:,3]
    M = y1[:,4]
    return I

#Uncomment extract2, extract3, result2 and result3 if you want to fit more than one curve at the same time
def comboFunc(comboData, b1, E0, I0):
    # single data set passed in, extract separate data
    extract1 = comboData[:len(y1)] # first data
#    extract2 = comboData[:len(y1)] # second data
#    extract3 = comboData[:len(y1)]
    
    result1 = mod1(extract1, b1, E0, I0)
#    result2 = mod2(extract2, b1, I0, N)
#    result3 = mod3(extract3, b1, E0, I0)
    
    return result1


# some initial parameter values
initialParameters = np.array([0.4, 5, 1])

# curve fit the combined data to the combined function
fittedParam, pcov = curve_fit(comboFunc, comboX, comboY, initialParameters, bounds = ((0,0,0), (1,1000,1000)))

# values for display of fitted function
b1, E0, I0 = fittedParam

y_fit_1 = mod1(t2, b1, E0, I0) # first data set, first equation
#y_fit_2 = mod2(t, b1, I0, N) # second data set, second equation
#y_fit_3 = mod3(t2, N, b1, E0, I0)

#plt.plot(comboX, comboY, 'b.') # plot the raw data if you're using more than one dataset
plt.plot(t2, y1, 'b.', label = 'Data')#plot of data
plt.plot(t2, y_fit_1 , label = 'Infected adjustment SEIRD', color = 'orange') # plot the equation using the fitted parameters
plt.xlabel('Days since 22/03/2020')
plt.ylabel('Number of people')
plt.legend(loc='best', fontsize = 10)

#Printing values for each parameter found and the chi^2
print('Chi^2:', r2_score(y1, y_fit_1))
print('b1 =', fittedParameters[0], '+/-', pcov[0,0]**0.5)
print('E0 =', fittedParameters[1], '+/-', pcov[1,1]**0.5)
print('I0 =', fittedParameters[2], '+/-', pcov[2,2]**0.5)


#mod4 function will be used for simulation of future scenarios given the parameters found by the fitting of mod1
def mod4(t, N, b1, E0, I0):
    def Corona(z,t):
        S = z[0]
        E = z[1]
        I = z[2]
        R = z[3]
        M = z[4]
        Pq = 0.5
        Pq2 = 0.2
        b = Pq*(b1/N)/(1+0.01*np.exp(t-8)) + (1-Pq-Pq2)*(b1/N) + Pq2*(b1/N)/(1+0.01*np.exp(0.2*(t-130)))  + (Pq+Pq2)*(b1/N)/(1+100*np.exp(-1*(t-190)))
        c = 1/5.1
        k = 0.44
        Pm = 0.007
        tr = 16
        a = (1-Pm)/tr
        tm = 13
        mu = Pm/tm
        dSdt = -(1-k)*b*I*S - k*b*E*S
        dEdt = (1-k)*b*I*S - c*E + k*b*E*S
        dIdt = c*E - a*I - mu*I
        dRdt = a*I
        dMdt = mu*I
        return [dSdt, dEdt, dIdt, dRdt, dMdt]
    
    
    #Defining initial conditions
    z0 = [N,E0,I0,0,0]
    y1 = odeint(Corona, z0, t)
    
    S = y1[:,0]
    E = y1[:,1]
    I = y1[:,2]
    R = y1[:,3]
    M = y1[:,4]
    CT = I + R + M
    return I


plt.rc('text', usetex = True)
plt.rc('font', family ='serif')

fig, ax1 = plt.subplots()

left, bottom, width, height = [0.68, 0.65, 0.2, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])

ax1.plot(t3, mod4(t3, Nmoç, fittedParam[0], fittedParam[1], fittedParam[2]), color = 'blue', label = 'N = 0.09\% of Nampula + Cabo Delgado + Maputo')
ax1.plot(t2, y1, 'k1', label = 'Data')

ax2.plot(t3[:350], mod4(t3, Nmoç, fittedParam[0], fittedParam[1], fittedParam[2])[:350], color = 'blue', label = 'N = 0.09\% of Nampula + Cabo Delgado + Maputo')
ax2.plot(t2[:350], y1[:200], 'k1', label = 'Data')

ax1.set_ylabel(r'Number of people')
ax1.set_xlabel('Days since 22/03/2020')
lgd = ax1.legend(bbox_to_anchor=(0.1, -0.15), loc='upper left', borderaxespad=0.)

Any doubts or comments/suggestions to the code please write an issue question, or e-mail pedrohpc96@hotmail.com with the subject GITHUB CODE MOZAMBIQUE